## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!{sys.executable} -m nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv10

**NOTE:** Currently, YOLOv10 does not have its own PyPI package. Therefore, we need to install the code from the source.

In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

**NOTE:** We will also install two additional packages: [`roboflow`](https://github.com/roboflow/roboflow-python) to download the dataset from [Roboflow Universe](https://universe.roboflow.com/), which we will use to train our model, and [`supervision`](https://github.com/roboflow/supervision), which we will use for visualizing the results.

In [ ]:
!pip install -q supervision roboflow

## Download pre-trained weights

**NOTE:** YOLOv10 provides weight files pre-trained on the COCO dataset in various sizes. Let's download them.

In [ ]:
!{sys.executable} -m mkdir -p {HOME}/weights
!{sys.executable} -m wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!{sys.executable} -m wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!{sys.executable} -m wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
# !{sys.executable} -m wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
# !{sys.executable} -m wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
# !{sys.executable} -m wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!{sys.executable} -m ls -lh {HOME}/weights

## Download example data

**NONE:** Let's download few example images. Feel free to use your images or videos.

In [ ]:
!{sys.executable} -m mkdir -p {HOME}/data
!{sys.executable} -m wget -P {HOME}/data -q https://media.roboflow.com/notebooks/examples/dog.jpeg
!{sys.executable} -m ls -lh {HOME}/data

## Inference with Pre-trained COCO Model

**NOTE:** YOLOv10 is based on YOLOv8, and like YOLOv8, it can be used in both CLI and SDK modes.

### 💻 CLI

In [ ]:
!python -V
!pip install huggingface huggingface_hub
%cd {HOME}

!yolo task=detect mode=predict conf=0.25 save=True \
model={HOME}/weights/yolov10n.pt \
source={HOME}/data/dog.jpeg

**NONE:** Let's display result.

In [ ]:
%cd {HOME}

from IPython.display import Image

Image(filename='runs/detect/predict/dog.jpeg', height=600)

### 🐍 Python SDK

In [ ]:
from ultralytics import YOLOv10

model = YOLOv10(f'{HOME}/weights/yolov10n.pt')
results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)
results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)
results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)
results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)
results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)
results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)

In [ ]:
results[0].boxes.xyxy

In [ ]:
results[0].boxes.conf

In [ ]:
results[0].boxes.cls

**NONE:** Let's display result using `supervision`.

In [ ]:
import cv2
import supervision as sv
from ultralytics import YOLOv10

model = YOLOv10(f'{HOME}/weights/yolov10n.pt')
image = cv2.imread(f'{HOME}/data/dog.jpeg')
results = model(image)[0]
detections = sv.Detections.from_ultralytics(results)

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

## Download dataset from Roboflow Universe

In [ ]:
!pip install pickleshare
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install -q roboflow

from roboflow import Roboflow


rf = Roboflow()
project = rf.workspace("selencakmak").project("tumor-dj2a1")
version = project.version(1)
dataset = version.download("yolov8")

**NOTE:**
Make sure the last 4 lines of the data.yaml file have the following format:

```
test: ../test/images
train: ../train/images
val: ../valid/images
```

If using a dataset from Roboflow Universe, run the command below. 👇🏻

In [ ]:
!{sys.executable} -m sed -i '$d' {dataset.location}/data.yaml
!{sys.executable} -m sed -i '$d' {dataset.location}/data.yaml
!{sys.executable} -m sed -i '$d' {dataset.location}/data.yaml
!{sys.executable} -m sed -i '$d' {dataset.location}/data.yaml
!{sys.executable} -m echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset.location}/data.yaml

## Custom Training

In [ ]:
!{sys.executable} -m yolo task=detect mode=train epochs=10 batch=2 plots=True \
  model=HOME/runs/detect/train10/weights/best.pt \
  data=HOME/datasets/weeds/data.yaml \
  optimizer=AdamW \
  verbose=True \
  augment=False


In [ ]:
!{sys.executable} -m ls {HOME}/runs/detect/train12/

In [ ]:
%cd {HOME}
from IPython.display import Image
Image(filename=f'{HOME}/runs/detect/train10/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train10/results.png', width=600)

## Inference with Custom Model

**NOTE:** Let's start by loading our newly trained model.

In [ ]:
from ultralytics import YOLOv10
import supervision as sv
model = YOLOv10(f'{HOME}/runs/detect/train10/weights/best.pt')

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="HOME/datasets/weeds/valid/images",
    annotations_directory_path="HOME/datasets/weeds/valid/labels",
    data_yaml_path="HOME/datasets/weeds/data.yaml"
)

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

**NOTE:** Let's randomly select an image from our validation set and visualize the results.

In [ ]:
import random

random_image = random.choice(list(dataset.images.keys()))
random_image = dataset.images[random_image]

results = model(source=random_image, conf=0.25)[0]
detections = sv.Detections.from_ultralytics(results)

annotated_image = bounding_box_annotator.annotate(
    scene=random_image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)